In [1]:
IMG_SHAPE=(224,224,3)
import tensorflow as tf
base_network = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')
base_network.trainable = False

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers 
import numpy as np
import tensorflow as tf
IMG_SHAPE=(224,224,3)
import cv2
import numpy as np
#from PIL import Image
import matplotlib.pyplot as plt
from skimage.transform import resize
path = "/content/drive/My Drive/MRI Bone/train/axial/"
path1 = "/content/drive/My Drive/MRI Bone/train-abnormal.csv"
import pandas as pd
lab_abn=pd.read_csv(path1)
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
feature_batch_averag=tf.zeros([1,1280])
feature_batch_average=tf.zeros([1,1280])
x = tf.placeholder('float', [None, 224,224,3])
xx = tf.placeholder('float', [None, 1280])
y = tf.placeholder('float')
feature_batch = base_network(x)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_averag = global_average_layer(feature_batch)
  #layer = tf.keras.layers.Dense(10, input_shape=(None, 1280))
keras.layers.Dropout(rate=0.2)
layer = tf.keras.layers.Dense(1,kernel_regularizer=regularizers.l2(0.001))

#layer = {'weights':tf.Variable(tf.random_normal([1280, 1])),
                    #'biases':tf.Variable(tf.random_normal([1])),}


#a = tf.matmul(xx,layer['weights']) + layer['biases']


a=layer(xx)
b = tf.keras.activations.sigmoid(a)
cost=tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=b)
  #cost = tf.reduce_mean( tf.nn.cross_entropy(logits=b, labels=y) )
optimizer = tf.train.AdamOptimizer().minimize(cost)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for epoch in range(40):
    epoch_loss=0
    for i in range(200):
    
      pad=str(i).zfill(4)
      path1=path+str(pad)+".npy"
      jj = np.load(path1)
      kk=np.zeros([256,256])
      ss=[]
      for img in range(jj.shape[0]):
  #for img in range(jj.shape[0]):
        kk=jj[img,:,:]
        kk = resize(kk, (224, 224),anti_aliasing=True)

        pp=np.zeros([256,256])
        vv=np.zeros([224,224])
        for ii in range(2):
          pp=jj[img,:,:]
          vv = resize(pp, (224, 224),anti_aliasing=True)
          
          vv = cv2.equalizeHist(vv.astype(np.uint8))
          kk=np.dstack((kk, vv))
    #print(kk.shape)
        kk = np.expand_dims(kk, axis=0)
    #print(kk= np.float32(kk))
    #print(kk.dtype)
    
        pk=(sess.run(feature_batch_averag, feed_dict={x: kk}))
        ss.append(pk[0])
      mm=max(ss,key=list)
      mm = np.expand_dims(mm, axis=0)
      _,c = sess.run([optimizer, cost], feed_dict={xx: mm, y: lab_abn.iloc[i,[1]]})
      epoch_loss += c
      print("round:",i )
    print("loss:",epoch_loss )
    #print(img,":",feature_batch_average)

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
9412608/9406464 [==============================] - 1s 0us/step


FileNotFoundError: ignored